### 1. 実験結果dfを作成

In [7]:
import pandas as pd

data = {
    'smile_time': [81, 174, 22, 312, 268, 225],  # 笑顔時間のリスト
    'ideas': [6, 4, 0, 3, 5, 5],  # アイデアの数のリスト
}

# DataFrameを作成
df = pd.DataFrame(data)

# DataFrameの表示
print(df)

### 2. Shapiro-Wilk検定（n<50の小規模サンプル向け正規性の検定)
参考：https://www.ibm.com/support/pages/kolmogorov-smirnov%E3%81%A8shapiro-wilk%E3%81%AE%E9%81%95%E3%81%84

In [8]:
from scipy.stats import shapiro

# 笑顔時間データ、アイデア数データなどを対象に正規性検定を実施
smile_time_data = df['smile_time']
ideas_data = df['ideas']

# 笑顔時間のShapiro-Wilk検定
stat_smile, p_value_smile = shapiro(smile_time_data)
print(f"smile_time p値: {p_value_smile}")
if p_value_smile > 0.05:
    print("笑顔時間のデータは正規分布に従う")
else:
    print("笑顔時間のデータは正規分布に従わない")

# アイデア数のShapiro-Wilk検定
stat_ideas, p_value_ideas = shapiro(ideas_data)
print(f"ideas p値: {p_value_ideas}")
if p_value_ideas > 0.05:
    print("アイデア数のデータは正規分布に従う")
else:
    print("アイデア数のデータは正規分布に従わない")

### 3. F検定（正規分布に従うときの2群間の等分散性の検定）
参考：https://datawokagaku.com/f_test/

In [9]:
import numpy as np
from scipy.stats import f

# 笑顔時間とアイデア数のデータ
smile_time = df['smile_time']
ideas = df['ideas']

# 分散を計算
var_smile_time = np.var(smile_time, ddof=1)  # 分散（自由度1を設定）
var_ideas = np.var(ideas, ddof=1)

# F検定のF値を計算
if var_smile_time > var_ideas:
    F = var_smile_time / var_ideas
else:
    F = var_ideas / var_smile_time

# 自由度を設定
dfn = len(smile_time) - 1  # 笑顔時間の自由度
dfd = len(ideas) - 1  # アイデア数の自由度

# p値を計算
p_value = 1 - f.cdf(F, dfn, dfd) # F分布の累積分布関数(cdf)を使ってp値を計算

print(f"F値: {F}")
print(f"p値: {p_value}")

if p_value < 0.05:
    print("帰無仮説を棄却: 等分散ではない")
else:
    print("帰無仮説を採択: 等分散が仮定できる")


### 4. Welchのt検定（等分散を仮定しない2群の平均値の比較）

In [10]:
from scipy.stats import ttest_ind

# 笑顔時間とアイデア数のデータを準備
smile_time = df['smile_time']  # 笑顔時間データ
ideas = df['ideas']  # アイデア数データ

# Welchのt検定の実施（等分散性を仮定しない場合）
stat, p_value = ttest_ind(smile_time, ideas, equal_var=False)

print(f"Welchのt検定の統計量: {stat}")
print(f"p値: {p_value}")

if p_value < 0.05:
    print("帰無仮説を棄却: 笑顔時間とアイデア数が影響し合っている可能性がある")
else:
    print("帰無仮説を採択: 笑顔時間とアイデア数は影響しあっていない")


### 5. 線形回帰分析

In [11]:
import statsmodels.api as sm

# データを準備
X = df['ideas']  # 独立変数: アイデア数
y = df['smile_time']  # 従属変数: 笑顔時間

# 定数項を追加
X = sm.add_constant(X)

# 回帰モデルのフィット
model = sm.OLS(y, X)
results = model.fit()

# 回帰結果の表示
# print(results.summary())

# 検定結果の解釈
p_value = results.pvalues[1]  # 独立変数 'ideas' に対応する p値
print(f"p値: {p_value}")

if p_value < 0.05:
    print("帰無仮説を棄却: アイデア数が笑顔時間に影響を与えている可能性がある")
else:
    print("帰無仮説を採択: アイデア数が笑顔時間に影響を与えていない")


In [12]:
# データを準備
X = df['smile_time']  # 独立変数: 笑顔時間
y = df['ideas']  # 従属変数: アイデア数

# 定数項を追加
X = sm.add_constant(X)

# 回帰モデルのフィット
model = sm.OLS(y, X)
results = model.fit()

# 回帰結果の表示
# print(results.summary())

# 検定結果の解釈
p_value = results.pvalues[1]  # 独立変数 'smile_time' に対応する p値
print(f"p値: {p_value}")

if p_value < 0.05:
    print("帰無仮説を棄却: 笑顔時間がアイデア数に影響を与えている可能性がある")
else:
    print("帰無仮説を採択: 笑顔時間がアイデア数に影響を与えていない")
